In [1]:
import pandas as pd
import sqlite3
from collections import Counter
import numpy as np

conn = sqlite3.connect("busdata_18.sqlite")
cur = conn.cursor()

q = "SELECT DISTINCT(LINEID) FROM trips_cleaned;"
lines = pd.read_sql_query(q, conn)

In [2]:
lines.head()

,LINEID
0,68
1,25B
2,45A
3,25A
4,14


In [3]:
unique_LINEID_list = lines.LINEID.tolist()

In [4]:
unique_LINEID_list.sort()

In [5]:
print(unique_LINEID_list)

['1', '102', '104', '11', '111', '114', '116', '118', '120', '122', '123', '13', '130', '14', '140', '142', '145', '14C', '15', '150', '151', '15A', '15B', '15D', '16', '161', '16C', '16D', '17', '17A', '18', '184', '185', '220', '236', '238', '239', '25', '25A', '25B', '25D', '25X', '26', '27', '270', '27A', '27B', '27X', '29A', '31', '31A', '31B', '31D', '32', '32X', '33', '33A', '33B', '33D', '33E', '33X', '37', '38', '38A', '38B', '38D', '39', '39A', '39X', '4', '40', '40B', '40D', '40E', '41', '41A', '41B', '41C', '41D', '41X', '42', '42D', '43', '44', '44B', '45A', '46A', '46E', '47', '49', '51D', '51X', '53', '54A', '56A', '59', '61', '63', '65', '65B', '66', '66A', '66B', '66X', '67', '67X', '68', '68A', '68X', '69', '69X', '7', '70', '70D', '75', '76', '76A', '77A', '77X', '79', '79A', '7A', '7B', '7D', '83', '83A', '84', '84A', '84X', '9']


In [6]:
def find_stops_per_route(LINEID):
    trips_result = cur.execute("SELECT DISTINCT(TRIPID) FROM trips_cleaned WHERE LINEID=?;", (LINEID,))
    trips_df = pd.DataFrame(trips_result.fetchall(), columns=["TRIPID"])
    unique_TRIPID_list = trips_df.TRIPID.tolist()
    print("Found trips for", LINEID)
    
    q = "SELECT DAYOFSERVICE, TRIPID, PROGRNUMBER, STOPPOINTID, PLANNEDTIME_ARR AS PLANNEDARR_STOP, PLANNEDTIME_DEP AS PLANNEDDEP_STOP, ACTUALTIME_ARR AS ACTUALARR_STOP, ACTUALTIME_DEP AS ACTUALDEP_STOP from leavetimes_cleaned where TRIPID in ({trip_list})".format(trip_list=','.join(["?"]*len(unique_TRIPID_list)))
    leavetimes_result = cur.execute(q, unique_TRIPID_list)
    line_df = pd.DataFrame(leavetimes_result.fetchall(), columns=["DAYOFSERVICE", "TRIPID", "PROGRNUMBER", "STOPPOINTID", "PLANNEDDEP_STOP", "PLANNEDARR_STOP", "ACTUALARR_STOP", "ACTUALDEP_STOP"])
    print("Found stops for", LINEID)
    line_df.to_csv("~/data/stoptimes/" + LINEID + "_stops.csv", index=False)

In [9]:
for line in unique_LINEID_list:
    find_stops_per_route(line)

Found trips for 1
Found stops for 1
Found trips for 102
Found stops for 102
Found trips for 104
Found stops for 104
Found trips for 11
Found stops for 11
Found trips for 111
Found stops for 111
Found trips for 114
Found stops for 114
Found trips for 116
Found stops for 116
Found trips for 118
Found stops for 118
Found trips for 120
Found stops for 120
Found trips for 122
Found stops for 122
Found trips for 123
Found stops for 123
Found trips for 13
Found stops for 13
Found trips for 130
Found stops for 130
Found trips for 14
Found stops for 14
Found trips for 140
Found stops for 140
Found trips for 142
Found stops for 142
Found trips for 145
Found stops for 145
Found trips for 14C
Found stops for 14C
Found trips for 15
Found stops for 15
Found trips for 150
Found stops for 150
Found trips for 151
Found stops for 151
Found trips for 15A
Found stops for 15A
Found trips for 15B
Found stops for 15B
Found trips for 15D
Found stops for 15D
Found trips for 16
Found stops for 16
Found trips fo

<h4>Find longest subroutes</h4>

In [36]:
q = "SELECT * FROM trips_cleaned;"
trips_df = pd.read_sql_query(q, conn)

In [7]:
trips_df.head()

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP
0,07-FEB-18 00:00:00,6253783,68,68_80,1,87245,84600,87524.0,84600.0
1,07-FEB-18 00:00:00,6262138,25B,25B_271,2,30517,26460,32752.0,NaN
2,07-FEB-18 00:00:00,6254942,45A,45A_70,2,35512,32100,36329.0,32082.0
3,07-FEB-18 00:00:00,6259460,25A,25A_273,1,57261,54420,58463.0,54443.0
4,07-FEB-18 00:00:00,6253175,14,14_15,1,85383,81600,84682.0,81608.0


In [27]:
longest_ob_subroutes = []

for line in unique_LINEID_list:   
    print(line)
    current_line_df = pd.read_csv("~/data/stoptimes/" + line + "_stops.csv")
    merged = current_line_df.merge(trips_df, how="left", on=["DAYOFSERVICE", "TRIPID"])
    
    ob_routes_df = merged.loc[merged["DIRECTION"] == 1]
    ob_routes_list = ob_routes_df.ROUTEID.unique().tolist()
    
    route_lengths = {}

    for route in ob_routes_list:
        current_route_df = ob_routes_df.loc[(ob_routes_df["ROUTEID"] == route)]
        num_stops = current_route_df.STOPPOINTID.nunique()
        route_lengths[route] = num_stops
        
    print(route_lengths)
    
    if route_lengths == {}:
        pass
    else:       
        max_length_route = max(route_lengths.items(), key=lambda x: x[1])
        routes = []

        for key, value in route_lengths.items():
            if value == max_length_route[1]:
                routes.append(key)
    
        print(routes)
        for route in routes:
            longest_ob_subroutes.append(route)
                

1
{'1_37': 42, '1_39': 23, '1_38': 21}
['1_37']
102
{'102_8': 53}
['102_8']
104
{'104_15': 36}
['104_15']
11
{'11_40': 61, '11_41': 21}
['11_40']
111
{'111_7': 46}
['111_7']
114
{'114_5': 29}
['114_5']
116
{'116_1': 50}
['116_1']
118
{}
120
{'120_7': 26, '120_8': 37, '120_12': 26}
['120_8']
122
{'122_14': 50, '122_15': 21, '122_18': 50, '122_19': 21}
['122_14', '122_18']
123
{'123_34': 44, '123_35': 29}
['123_34']
13
{'13_60': 90, '13_65': 87, '13_61': 91, '13_66': 88, '13_63': 91, '13_64': 33, '13_62': 90, '13_28': 32}
['13_61', '13_63']
130
{'130_10': 39}
['130_10']
14
{'14_15': 76}
['14_15']
140
{'140_19': 44, '140_20': 27, '140_16': 26}
['140_19']
142
{'142_8': 20, '142_10': 53, '142_13': 55}
['142_13']
145
{'145_102': 70, '145_95': 23, '145_104': 51, '145_94': 60, '145_99': 48, '145_107': 64}
['145_102']
14C
{'14C_17': 30}
['14C_17']
15
{'15_16': 71, '15_10': 35, '15_19': 70, '15_20': 34}
['15_16']
150
{'150_8': 35}
['150_8']
151
{'151_15': 50, '151_16': 50}
['151_15', '151_16']
1

In [28]:
print(longest_ob_subroutes)

['1_37', '102_8', '104_15', '11_40', '111_7', '114_5', '116_1', '120_8', '122_14', '122_18', '123_34', '13_61', '13_63', '130_10', '14_15', '140_19', '142_13', '145_102', '14C_17', '15_16', '150_8', '151_15', '151_16', '15A_83', '15B_60', '15B_56', '15D_62', '16_20', '161_50', '16C_29', '16D_30', '17_10', '17A_11', '17A_15', '18_3', '184_29', '185_55', '185_54', '220_14', '236_9', '238_11', '239_26', '25_272', '25A_273', '25B_274', '25D_277', '25X_10', '26_24', '26_28', '27_19', '270_42', '27A_4', '27B_23', '27X_43', '29A_14', '31_17', '31A_25', '31B_44', '31D_50', '32_57', '32X_74', '32X_77', '33_69', '33A_22', '33B_56', '33D_61', '33E_74', '33X_46', '37_14', '38_10', '38A_20', '38B_40', '38D_68', '39_22', '39A_40', '39X_1', '4_10', '40_27', '40B_63', '40D_101', '40E_90', '41_22', '41B_58', '41C_78', '41C_83', '41D_26', '41X_121', '42_42', '42D_50', '43_85', '43_84', '44_40', '44B_60', '45A_67', '46A_74', '47_138', '49_27', '51D_50', '53_22', '54A_11', '56A_31', '59_10', '61_107', '63

In [29]:
file = open("stoptimes/longest_ob_subroutes.txt", "w")
for element in longest_ob_subroutes:
    file.write(element + "\n")
file.close()

print("Write successful")

Write successful


In [30]:
longest_ib_subroutes = []

for line in unique_LINEID_list:   
    print(line)
    current_line_df = pd.read_csv("~/data/stoptimes/" + line + "_stops.csv")
    merged = current_line_df.merge(trips_df, how="left", on=["DAYOFSERVICE", "TRIPID"])
    
    ib_routes_df = merged.loc[merged["DIRECTION"] == 2]
    ib_routes_list = ib_routes_df.ROUTEID.unique().tolist()
    
    route_lengths = {}

    for route in ib_routes_list:
        current_route_df = ib_routes_df.loc[(ib_routes_df["ROUTEID"] == route)]
        num_stops = current_route_df.STOPPOINTID.nunique()
        route_lengths[route] = num_stops
        
    print(route_lengths)
    
    if route_lengths == {}:
        pass
    else:       
        max_length_route = max(route_lengths.items(), key=lambda x: x[1])
        routes = []

        for key, value in route_lengths.items():
            if value == max_length_route[1]:
                routes.append(key)
    
        print(routes)
        for route in routes:
            longest_ib_subroutes.append(route)

1
{'1_40': 43, '1_41': 22}
['1_40']
102
{'102_10': 31, '102_9': 54}
['102_9']
104
{'104_16': 36}
['104_16']
11
{'11_42': 66, '11_43': 43}
['11_42']
111
{'111_8': 48, '111_9': 48, '111_10': 15}
['111_8', '111_9']
114
{'114_6': 29}
['114_6']
116
{'116_3': 65}
['116_3']
118
{'118_3': 43, '118_4': 44}
['118_4']
120
{'120_9': 24, '120_10': 34, '120_11': 34, '120_13': 24}
['120_10', '120_11']
122
{'122_16': 52, '122_17': 30, '122_20': 52, '122_21': 30}
['122_16', '122_20']
123
{'123_36': 47, '123_37': 17}
['123_36']
13
{'13_67': 90, '13_71': 55, '13_68': 90, '13_73': 87, '13_76': 54, '13_70': 90, '13_69': 89, '13_72': 86, '13_75': 36, '13_74': 87}
['13_67', '13_68', '13_70']
130
{'130_11': 38}
['130_11']
14
{'14_16': 75, '14_12': 30, '14_13': 69}
['14_16']
140
{'140_21': 47, '140_22': 19}
['140_21']
142
{'142_9': 17, '142_12': 49}
['142_12']
145
{'145_105': 74, '145_91': 58, '145_93': 74, '145_88': 65, '145_106': 69, '145_89': 39, '145_87': 22, '145_86': 27, '145_108': 49}
['145_105', '145_9

In [31]:
print(longest_ib_subroutes)

['1_40', '102_9', '104_16', '11_42', '111_8', '111_9', '114_6', '116_3', '118_4', '120_10', '120_11', '122_16', '122_20', '123_36', '13_67', '13_68', '13_70', '130_11', '14_16', '140_21', '142_12', '145_105', '145_93', '14C_18', '15_17', '150_9', '151_17', '15A_84', '15B_61', '15D_63', '16_24', '161_51', '16C_28', '17_15', '17A_13', '17A_17', '18_4', '184_28', '185_59', '185_62', '185_60', '220_15', '236_10', '238_15', '239_28', '25_269', '25A_270', '25B_271', '25D_275', '25X_9', '26_26', '26_29', '27_17', '270_44', '27A_7', '27B_34', '27X_42', '29A_15', '31_18', '31A_26', '31B_46', '31D_51', '32_58', '32X_76', '33_70', '33A_25', '33B_58', '33D_62', '33X_47', '37_15', '38_15', '38A_33', '38B_44', '38D_61', '39_21', '39A_43', '39X_2', '39X_3', '4_12', '4_14', '40_31', '40B_65', '40D_103', '40E_91', '41_25', '41A_22', '41B_53', '41C_79', '41D_27', '41X_128', '41X_126', '42_44', '42D_51', '43_89', '43_88', '44_36', '44B_61', '45A_69', '46A_67', '46E_93', '46E_94', '47_135', '49_29', '51D_

In [32]:
file = open("stoptimes/longest_ib_subroutes.txt", "w")
for element in longest_ib_subroutes:
    file.write(element + "\n")
file.close()

print("Write successful")

Write successful


In [8]:
with open("stoptimes/longest_ob_subroutes.txt") as file:
    longest_ob_subroutes = file.readlines()
longest_ob_subroutes = [line.strip() for line in longest_ob_subroutes] 

In [9]:
with open("stoptimes/longest_ib_subroutes.txt") as file:
    longest_ib_subroutes = file.readlines()
longest_ib_subroutes = [line.strip() for line in longest_ib_subroutes] 

In [35]:
def get_stop_sequence(route_list):
    stop_sequence_all_routes = pd.DataFrame(columns=["LINEID", "ROUTEID", "DIRECTION", "PROGRNUMBER", "STOPPOINTID"])

    for route in route_list:
        line = route.split("_")[0]
        current = pd.read_csv("~/data/stoptimes/" + line + "_stops.csv")
        merged = current.merge(trips_df, how="left", on=["DAYOFSERVICE", "TRIPID"])
        merged = merged.loc[merged["ROUTEID"] == route] 
        merged = merged.sort_values(by=["PROGRNUMBER"])
        stop_sequence = merged[["LINEID", "ROUTEID", "DIRECTION", "PROGRNUMBER", "STOPPOINTID"]].copy()
        stop_sequence = stop_sequence.drop_duplicates()
        stop_sequence_all_routes = stop_sequence_all_routes.append(stop_sequence)
        print("Route", route, "complete")
    return stop_sequence_all_routes

In [36]:
ob_subroutes_df = get_stop_sequence(longest_ob_subroutes)
ob_subroutes_df.shape

Route 1_37 complete
Route 102_8 complete
Route 104_15 complete
Route 11_40 complete
Route 111_7 complete
Route 114_5 complete
Route 116_1 complete
Route 120_8 complete
Route 122_14 complete
Route 122_18 complete
Route 123_34 complete
Route 13_61 complete
Route 13_63 complete
Route 130_10 complete
Route 14_15 complete
Route 140_19 complete
Route 142_13 complete
Route 145_102 complete
Route 14C_17 complete
Route 15_16 complete
Route 150_8 complete
Route 151_15 complete
Route 151_16 complete
Route 15A_83 complete
Route 15B_60 complete
Route 15B_56 complete
Route 15D_62 complete
Route 16_20 complete
Route 161_50 complete
Route 16C_29 complete
Route 16D_30 complete
Route 17_10 complete
Route 17A_11 complete
Route 17A_15 complete
Route 18_3 complete
Route 184_29 complete
Route 185_55 complete
Route 185_54 complete
Route 220_14 complete
Route 236_9 complete
Route 238_11 complete
Route 239_26 complete
Route 25_272 complete
Route 25A_273 complete
Route 25B_274 complete
Route 25D_277 complete
Ro

(7924, 5)

In [37]:
ob_subroutes_df.head()

,LINEID,ROUTEID,DIRECTION,PROGRNUMBER,STOPPOINTID
512465,1,1_37,1,1,226
176654,1,1_37,1,2,228
1278020,1,1_37,1,3,229
554384,1,1_37,1,4,227
785345,1,1_37,1,5,230


In [38]:
ob_subroutes_df.isna().sum()

LINEID         0
ROUTEID        0
DIRECTION      0
PROGRNUMBER    0
STOPPOINTID    0
dtype: int64

In [39]:
ib_subroutes_df = get_stop_sequence(longest_ib_subroutes)

Route 1_40 complete
Route 102_9 complete
Route 104_16 complete
Route 11_42 complete
Route 111_8 complete
Route 111_9 complete
Route 114_6 complete
Route 116_3 complete
Route 118_4 complete
Route 120_10 complete
Route 120_11 complete
Route 122_16 complete
Route 122_20 complete
Route 123_36 complete
Route 13_67 complete
Route 13_68 complete
Route 13_70 complete
Route 130_11 complete
Route 14_16 complete
Route 140_21 complete
Route 142_12 complete
Route 145_105 complete
Route 145_93 complete
Route 14C_18 complete
Route 15_17 complete
Route 150_9 complete
Route 151_17 complete
Route 15A_84 complete
Route 15B_61 complete
Route 15D_63 complete
Route 16_24 complete
Route 161_51 complete
Route 16C_28 complete
Route 17_15 complete
Route 17A_13 complete
Route 17A_17 complete
Route 18_4 complete
Route 184_28 complete
Route 185_59 complete
Route 185_62 complete
Route 185_60 complete
Route 220_15 complete
Route 236_10 complete
Route 238_15 complete
Route 239_28 complete
Route 25_269 complete
Route 

In [41]:
ib_subroutes_df.shape

(8293, 5)

In [42]:
ib_subroutes_df.head()

,LINEID,ROUTEID,DIRECTION,PROGRNUMBER,STOPPOINTID
1096219,1,1_40,2,1,381
758662,1,1_40,2,2,382
1261516,1,1_40,2,3,4451
1217980,1,1_40,2,4,383
1151892,1,1_40,2,5,384


In [48]:
ib_subroutes_df.isna().sum()

LINEID         0
ROUTEID        0
DIRECTION      0
PROGRNUMBER    0
STOPPOINTID    0
dtype: int64

In [43]:
stops_per_longest_subroutes = ob_subroutes_df.append(ib_subroutes_df)

In [44]:
stops_per_longest_subroutes.shape

(16217, 5)

In [51]:
stops_per_longest_subroutes.drop_duplicates(inplace=True)
stops_per_longest_subroutes.shape

(16217, 5)

In [54]:
stops_per_longest_subroutes.to_csv("stoptimes/stops_per_longest_subroutes.csv")

In [10]:
stops_per_longest_subroutes = pd.read_csv("stoptimes/stops_per_longest_subroutes.csv")

In [12]:
stops_per_longest_subroutes.drop(stops_per_longest_subroutes.columns[0], axis=1, inplace=Tr

In [13]:
stops_per_longest_subroutes.head(1)

,LINEID,ROUTEID,DIRECTION,PROGRNUMBER,STOPPOINTID
0,1,1_37,1,1,226


<h4>Find lines that have multiple "longest" subroutes</h4>

In [14]:
# Find lines that have multiple longest subroutes (outbound)
line_duplicates = []

for route in longest_ob_subroutes:
    line = route.split("_")[0]
    line_duplicates.append(line)

count = Counter(line_duplicates)

multiple_ob_subroutes = list([item for item in count if count[item] > 1])
print(multiple_ob_subroutes)

['122', '13', '151', '15B', '17A', '185', '26', '32X', '41C', '43', '66', '66A', '66B', '68', '69', '7', '77A']


In [15]:
# Find lines that have multiple longest subroutes (inbound)
line_duplicates = []

for route in longest_ib_subroutes:
    line = route.split("_")[0]
    line_duplicates.append(line)

count = Counter(line_duplicates)

multiple_ib_subroutes = list([item for item in count if count[item] > 1])
print(multiple_ib_subroutes)

['111', '120', '122', '13', '145', '17A', '185', '26', '39X', '4', '41X', '43', '46E', '51D', '66', '66A', '66B', '69', '7', '77A', '77X', '7A']


<h4>Check if stops match where there is more than one "longest" subroute for outbound routes</h4>

In [25]:
# Check if stops match where there is more than one longest subroute
for line in multiple_ob_subroutes:
    current = stops_per_longest_subroutes.loc[(stops_per_longest_subroutes["LINEID"] == line) & (stops_per_longest_subroutes["DIRECTION"] == 1)]
    routes = current.ROUTEID.unique().tolist()
    route_dict = {}
    for route in routes:
        current_route = current.loc[current["ROUTEID"] == route]
        unique_stops = current_route.STOPPOINTID.unique().tolist()
        route_dict[route] = unique_stops
    
    expected_stops = next(iter(route_dict.values()))
    all_equal = all(value == expected_stops for value in route_dict.values())
    
    if all_equal:
        print(line, "ok")
    else:
        print("Mismatch for line", line, "outbound")

122 ok
Mismatch for line 13 outbound
151 ok
15B ok
17A ok
185 ok
26 ok
Mismatch for line 32X outbound
41C ok
Mismatch for line 43 outbound
66 ok
66A ok
66B ok
Mismatch for line 68 outbound
69 ok
Mismatch for line 7 outbound
77A ok


In [85]:
def check_mismatched_stops(line, direction):
    subsection = stops_per_longest_subroutes.loc[(stops_per_longest_subroutes["LINEID"] == line) & (stops_per_longest_subroutes["DIRECTION"] == direction)]
    list_routes = subsection.ROUTEID.unique().tolist()
        
    all_subsets = []
    
    for route in list_routes:
        list_stops = subsection.loc[subsection["ROUTEID"] == route].STOPPOINTID.unique().tolist()
        all_subsets.extend(list_stops)
        
    problem_stops = [stop for stop in set(all_subsets) if all_subsets.count(stop) <= 1]
    return problem_stops


In [86]:
ob_13 = check_mismatched_stops("13", 1)
ob_13

[7229, 7230]

In [87]:
ob_32 = check_mismatched_stops("32X", 1)
ob_32

[4495, 403, 2809, 2810, 333, 747]

In [88]:
ob_43 = check_mismatched_stops("43", 1)
ob_43

[7189, 1199, 4917, 4918, 4919, 6359]

In [89]:
ob_68 = check_mismatched_stops("68", 1)
ob_68

[7483, 7547]

In [90]:
ob_7 = check_mismatched_stops("7", 1)
ob_7

[1174, 4962]

<h4>Check if stops match where there is more than one "longest" subroute for outbound routes</h4>

In [93]:
# Check if stops match where there is more than one longest subroute
for line in multiple_ib_subroutes:
    current = stops_per_longest_subroutes.loc[(stops_per_longest_subroutes["LINEID"] == line) & (stops_per_longest_subroutes["DIRECTION"] == 2)]
    routes = current.ROUTEID.unique().tolist()
    route_dict = {}
    for route in routes:
        current_route = current.loc[current["ROUTEID"] == route]
        unique_stops = current_route.STOPPOINTID.unique().tolist()
        route_dict[route] = unique_stops
    
    expected_stops = next(iter(route_dict.values()))
    all_equal = all(value == expected_stops for value in route_dict.values())
    
    if all_equal:
        print(line, "ok")
    else:
        print("Mismatch for line", line, "inbound")

Mismatch for line 111 inbound
Mismatch for line 120 inbound
122 ok
Mismatch for line 13 inbound
Mismatch for line 145 inbound
17A ok
185 ok
26 ok
Mismatch for line 39X inbound
4 ok
Mismatch for line 41X inbound
Mismatch for line 43 inbound
46E ok
Mismatch for line 51D inbound
66 ok
66A ok
66B ok
69 ok
7 ok
77A ok
77X ok
7A ok
